In [1]:
from core.board import Board
from analysis.msm_builder import get_msm_graph
from solver.solver import solve

In [2]:
r = 2
c = 5
n = 4
start = (0,0)
seed = None          
preset = True
profile = False
B_t = Board.random_complete(r,c,n,exclude=[start],seed=seed)
B_k = Board(r,c,n)
if preset:
    B_k.set_clear(0,0,1)
    B_k.set_clear(1,0,1)
    B_k.set_clear(1,3,3)
    B_k.set_mine(1,2)
    B_k.set_mine(1,4)
    pass
if preset: print(B_k)
elif seed is not None: print(B_t)

-------2/4
|1    |
|1 *3*|
-------


In [3]:
from analysis.msm_analysis import find_solutions, seperate_connected_msm_components
from analysis.probability import calculate_probabilities
from analysis.ordered_probability import calculate_safety
if preset:
    B_o, msm, c, f = get_msm_graph(B_k, order=2, verbose=0)
    #print(B_o, msm, c, f)
    msms = seperate_connected_msm_components(msm)
    #print(solutions := find_solutions(msms[0], verbose=0))
    #print(solutions.get_solution_counts_with_coords(clear_coords=[(0,1)]))
    #print(solutions.get_solution_counts_for_numbers((0,1), clear_coords=[(0,1)]))
    print(calculate_safety(B_k, order=1, prune_threshold=0, verbose=3))

Size of MSM Graph: 2
number of solutions for component
[[0 1 0 0 0]
 [0 1 0 0 0]]: {1: 2}
number of solutions for component
[[0 0 1 1 1]
 [0 0 0 0 0]]: {1: 3}
number of total solutions: {2: 6}
number of mines to place: 2
total possibilities: 6
computing probability at (0, 1)
if mine at (0, 1), component counts: {1: 1}
number of non-complement solutions given mine at (0, 1): {2: 3}
possibilities if mine at (0, 1): 3
computing probability at (1, 1)
if mine at (1, 1), component counts: {1: 1}
number of non-complement solutions given mine at (1, 1): {2: 3}
possibilities if mine at (1, 1): 3
computing probability at (0, 2)
if mine at (0, 2), component counts: {1: 2}
number of non-complement solutions given mine at (0, 2): {2: 4}
possibilities if mine at (0, 2): 4
computing probability at (0, 3)
if mine at (0, 3), component counts: {1: 2}
number of non-complement solutions given mine at (0, 3): {2: 4}
possibilities if mine at (0, 3): 4
computing probability at (0, 4)
if mine at (0, 4), compo

In [4]:
import cProfile
if not preset and seed is not None:
    B_k = Board(r,c,n)
    if profile:
        cProfile.run('print(solve(B_k, B_t, first_move=start, verbose=0))')
    else:
        print(solve(B_k, B_t, first_move=start, verbose=2))

In [5]:
import random
if seed is None and not preset:
    start = (0,0)
    max_iterations = 1000
    max_wins = 200
    wins = 0
    win_time = 0
    win_guesses = 0
    win_risk = 0
    ng_games = 0
    i = 0
    for i in range(max_iterations):
        seed = random.randint(0, 2**32)
        B_t = Board.random_complete(r,c,n,exclude=[start], seed=seed)
        B_k = Board(r,c,n)
        print(seed, end=' ')
        win, stats = solve(B_k, B_t, first_move=start, verbose=0)
        if win: 
            wins += 1
            win_time += stats.time
            win_guesses += stats.guesses
            win_risk += stats.risk
            if stats.guesses == 0: ng_games += 1
        print(win, f'{wins}/{i+1}', stats)
        if wins >= max_wins: break
    seed = None
    i += 1
    print(f'Wins: {wins/i}, Avg_Time: {win_time/wins}, Avg_guess: {win_guesses/wins}, Avg_risk:{win_risk/wins}, NG: {ng_games}')